In [21]:
import pandas as pd
import numpy as np

In [25]:
df=pd.read_csv('/rds/general/user/hsl121/home/hda_project/rq1/rq1_cleaned_no_ae.csv')
adverse=pd.read_csv('/rds/general/user/hsl121/home/hda_project/rq1/rq1_full_df_adverse.csv')

In [23]:
df.shape

(20593, 106)

Binary AE Added

In [26]:
df = df.merge(adverse[['SID', 'adverse_binary']], on='SID', how='left')

In [27]:
df.shape

(20593, 107)